In [3]:
# Importación de librerías esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import json
from scipy import stats

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Entorno configurado correctamente")
print(f"Versión de pandas: {pd.__version__}")
print(f"Versión de numpy: {np.__version__}")

Entorno configurado correctamente
Versión de pandas: 2.2.3
Versión de numpy: 2.2.0


#📌 Extracción

In [4]:
# URL de la API de datos
api_url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Extracción de datos desde API
try:
    df_raw = pd.read_json(api_url)
    print(f"Datos cargados exitosamente")
    print(f"Dimensiones del dataset: {df_raw.shape}")
    print(f"Columnas principales: {list(df_raw.columns)}")
except Exception as e:
    print(f"Error al cargar datos: {e}")

# Exploración de la estructura
print("\nPrimeras 3 filas del dataset:")
df_raw.head(3)

Datos cargados exitosamente
Dimensiones del dataset: (7267, 6)
Columnas principales: ['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']

Primeras 3 filas del dataset:


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}","{'Contract': 'One year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}"
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 9}","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'No', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'Yes'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'No', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 59.9, 'Total': '542.4'}}"
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 4}","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecurity': 'No', 'OnlineBackup': 'No', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'No', 'StreamingMovies': 'No'}","{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Electronic check', 'Charges': {'Monthly': 73.9, 'Total': '280.85'}}"


In [5]:
# Análisis de la estructura jerárquica
print("Estructura de datos anidados:")
for col in df_raw.columns:
    if isinstance(df_raw[col].iloc[0], dict):
        print(f"\n{col} (diccionario):")
        sample_dict = df_raw[col].iloc[0]
        for key in sample_dict.keys():
            print(f"  - {key}: {type(sample_dict[key]).__name__}")
    else:
        print(f"{col}: {type(df_raw[col].iloc[0]).__name__}")

Estructura de datos anidados:
customerID: str
Churn: str

customer (diccionario):
  - gender: str
  - SeniorCitizen: int
  - Partner: str
  - Dependents: str
  - tenure: int

phone (diccionario):
  - PhoneService: str
  - MultipleLines: str

internet (diccionario):
  - InternetService: str
  - OnlineSecurity: str
  - OnlineBackup: str
  - DeviceProtection: str
  - TechSupport: str
  - StreamingTV: str
  - StreamingMovies: str

account (diccionario):
  - Contract: str
  - PaperlessBilling: str
  - PaymentMethod: str
  - Charges: dict


#🔧 Transformación

In [6]:
# Normalización de estructuras anidadas
print("Normalizando estructuras de datos anidadas...")

if 'customer' in df_raw.columns:
    customer_df = pd.json_normalize(df_raw['customer'])
    print(f"Variables de customer: {list(customer_df.columns)}")

if 'phone' in df_raw.columns:
    phone_df = pd.json_normalize(df_raw['phone'])
    print(f"Variables de phone: {list(phone_df.columns)}")

if 'internet' in df_raw.columns:
    internet_df = pd.json_normalize(df_raw['internet'])
    print(f"Variables de internet: {list(internet_df.columns)}")

if 'account' in df_raw.columns:
    account_df = pd.json_normalize(df_raw['account'])
    print(f"Variables de account: {list(account_df.columns)}")

df = pd.concat([
    df_raw[['customerID', 'Churn']],
    customer_df,
    phone_df,
    internet_df,
    account_df
], axis=1)

print(f"\nDataset normalizado - Dimensiones: {df.shape}")
print(f"Total de variables: {len(df.columns)}")

Normalizando estructuras de datos anidadas...
Variables de customer: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure']
Variables de phone: ['PhoneService', 'MultipleLines']
Variables de internet: ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
Variables de account: ['Contract', 'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly', 'Charges.Total']

Dataset normalizado - Dimensiones: (7267, 21)
Total de variables: 21


In [7]:
# Análisis de valores únicos por columna
print("Análisis de valores únicos por variable:")
unique_analysis = pd.DataFrame({
    'Variable': df.columns,
    'Tipo': df.dtypes,
    'Valores_Unicos': [df[col].nunique() for col in df.columns],
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})

unique_analysis.sort_values('Valores_Unicos', ascending=False)

Análisis de valores únicos por variable:


,Variable,Tipo,Valores_Unicos,Valores_Nulos,Porcentaje_Nulos
customerID,customerID,object,7267,0,0.0
Charges.Total,Charges.Total,object,6531,0,0.0
Charges.Monthly,Charges.Monthly,float64,1585,0,0.0
tenure,tenure,int64,73,0,0.0
PaymentMethod,PaymentMethod,object,4,0,0.0
Contract,Contract,object,3,0,0.0
StreamingMovies,StreamingMovies,object,3,0,0.0
DeviceProtection,DeviceProtection,object,3,0,0.0
Churn,Churn,object,3,0,0.0
OnlineBackup,OnlineBackup,object,3,0,0.0


In [8]:
# Limpieza de la variable objetivo (Churn)
print("Limpieza de variable objetivo (Churn):")
print(f"Valores únicos en Churn antes de limpieza: {df['Churn'].unique()}")
print(f"Conteo de valores: {df['Churn'].value_counts()}")

registros_iniciales = len(df)
df = df[df['Churn'].notna()]
df = df[df['Churn'] != '']
df = df[df['Churn'].str.strip() != '']

registros_finales = len(df)
registros_eliminados = registros_iniciales - registros_finales

print(f"\nRegistros eliminados por Churn vacío: {registros_eliminados}")
print(f"Registros restantes: {registros_finales}")
print(f"Valores únicos en Churn después de limpieza: {df['Churn'].unique()}")

Limpieza de variable objetivo (Churn):
Valores únicos en Churn antes de limpieza: ['No' 'Yes' '']
Conteo de valores: Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

Registros eliminados por Churn vacío: 224
Registros restantes: 7043
Valores únicos en Churn después de limpieza: ['No' 'Yes']


In [9]:
# Reemplazo de strings vacíos con NaN
print("Reemplazando strings vacíos con NaN...")
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Análisis de valores faltantes después de limpieza
print("\nAnálisis de valores faltantes:")
missing_analysis = pd.DataFrame({
    'Variable': df.columns,
    'Valores_Faltantes': df.isnull().sum(),
    'Porcentaje': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_analysis = missing_analysis[missing_analysis['Valores_Faltantes'] > 0]
missing_analysis.sort_values('Porcentaje', ascending=False)

Reemplazando strings vacíos con NaN...

Análisis de valores faltantes:


,Variable,Valores_Faltantes,Porcentaje
Charges.Total,Charges.Total,11,0.16


In [10]:
# Identificación y clasificación de variables por tipo
print("Clasificación de variables por tipo:")

# Variables categóricas binarias (Yes/No)
binary_vars = []
for col in df.columns:
    if df[col].dtype == 'object':
        unique_vals = df[col].dropna().unique()
        if len(unique_vals) == 2 and set(unique_vals).issubset({'Yes', 'No'}):
            binary_vars.append(col)

print(f"Variables binarias (Yes/No): {binary_vars}")

# Variables categóricas múltiples
categorical_vars = []
for col in df.columns:
    if df[col].dtype == 'object' and col not in binary_vars and col not in ['customerID']:
        categorical_vars.append(col)

print(f"Variables categóricas múltiples: {categorical_vars}")

# Variables numéricas
numeric_vars = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variables numéricas: {numeric_vars}")

Clasificación de variables por tipo:
Variables binarias (Yes/No): ['Churn', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
Variables categóricas múltiples: ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod', 'Charges.Total']
Variables numéricas: ['SeniorCitizen', 'tenure', 'Charges.Monthly']


In [11]:
# Conversión de variables binarias Yes/No a 1/0
print("Convirtiendo variables binarias Yes/No a 1/0...")
for col in binary_vars:
    df[col] = df[col].map({'Yes': 1, 'No': 0})
    print(f"{col}: {df[col].value_counts().to_dict()}")

# Manejo especial de variables con "No internet service" o "No phone service"
print("\nManejando valores especiales...")
special_replacements = {
    'No internet service': 0,
    'No phone service': 0
}

for col in df.columns:
    if df[col].dtype == 'object':
        for old_val, new_val in special_replacements.items():
            if old_val in df[col].values:
                df[col] = df[col].replace(old_val, new_val)
                print(f"Reemplazado '{old_val}' con {new_val} en {col}")

Convirtiendo variables binarias Yes/No a 1/0...
Churn: {0: 5174, 1: 1869}
Partner: {0: 3641, 1: 3402}
Dependents: {0: 4933, 1: 2110}
PhoneService: {1: 6361, 0: 682}
PaperlessBilling: {1: 4171, 0: 2872}

Manejando valores especiales...
Reemplazado 'No phone service' con 0 en MultipleLines
Reemplazado 'No internet service' con 0 en OnlineSecurity
Reemplazado 'No internet service' con 0 en OnlineBackup
Reemplazado 'No internet service' con 0 en DeviceProtection
Reemplazado 'No internet service' con 0 en TechSupport
Reemplazado 'No internet service' con 0 en StreamingTV
Reemplazado 'No internet service' con 0 en StreamingMovies


In [ ]:
# Conversión de variables numéricas con manejo de errores
print("Convirtiendo variables numéricas...")

potential_numeric = ['tenure', 'MonthlyCharges', 'TotalCharges']

for col in potential_numeric:
    if col in df.columns:
        print(f"\nProcesando {col}:")
        print(f"Tipo actual: {df[col].dtype}")
        print(f"Valores únicos (primeros 10): {df[col].unique()[:10]}")
        
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        nan_count = df[col].isnull().sum()
        print(f"Valores convertidos a NaN: {nan_count}")
        
        if col == 'TotalCharges' and nan_count > 0:
            df[col].fillna(0, inplace=True)
            print(f"NaN en {col} rellenados con 0")

Convirtiendo variables numéricas...

Procesando tenure:
Tipo actual: int64
Valores únicos (primeros 10): [ 9  4 13  3 71 63  7 65 54 72]
Valores convertidos a NaN: 0


In [13]:
# Creación de la variable cuentas_diarias
print("Creando variable derivada: cuentas_diarias")

if 'MonthlyCharges' in df.columns:
    df['cuentas_diarias'] = (df['MonthlyCharges'] / 30).round(2)
    print(f"Variable cuentas_diarias creada")
    print(f"Estadísticas de cuentas_diarias:")
    print(df['cuentas_diarias'].describe())

if 'tenure' in df.columns:
    print("\nCreando segmentación por tenure:")
    df['tenure_segment'] = pd.cut(df['tenure'], 
                                 bins=[0, 12, 24, 48, 100], 
                                 labels=['0-12 meses', '13-24 meses', '25-48 meses', '48+ meses'],
                                 include_lowest=True)
    print(f"Distribución por segmento de tenure:")
    print(df['tenure_segment'].value_counts())

if 'MonthlyCharges' in df.columns:
    print("\nCreando segmentación por costo mensual:")
    df['cost_segment'] = pd.cut(df['MonthlyCharges'], 
                               bins=[0, 35, 65, 100, 200], 
                               labels=['Bajo', 'Medio', 'Alto', 'Premium'],
                               include_lowest=True)
    print(f"Distribución por segmento de costo:")
    print(df['cost_segment'].value_counts())

Creando variable derivada: cuentas_diarias

Creando segmentación por tenure:
Distribución por segmento de tenure:
tenure_segment
48+ meses      2239
0-12 meses     2186
25-48 meses    1594
13-24 meses    1024
Name: count, dtype: int64


In [14]:
# Diccionario de mapeo para nombres en español
column_mapping = {
    'customerID': 'id_cliente',
    'Churn': 'abandono',
    'gender': 'genero',
    'SeniorCitizen': 'mayor_65',
    'Partner': 'tiene_pareja',
    'Dependents': 'tiene_dependientes',
    'tenure': 'antiguedad_meses',
    'PhoneService': 'servicio_telefono',
    'MultipleLines': 'lineas_multiples',
    'InternetService': 'servicio_internet',
    'OnlineSecurity': 'seguridad_online',
    'OnlineBackup': 'respaldo_online',
    'DeviceProtection': 'proteccion_dispositivo',
    'TechSupport': 'soporte_tecnico',
    'StreamingTV': 'streaming_tv',
    'StreamingMovies': 'streaming_peliculas',
    'Contract': 'tipo_contrato',
    'PaperlessBilling': 'facturacion_digital',
    'PaymentMethod': 'metodo_pago',
    'MonthlyCharges': 'costo_mensual',
    'TotalCharges': 'costo_total'
}

existing_mappings = {k: v for k, v in column_mapping.items() if k in df.columns}
df.rename(columns=existing_mappings, inplace=True)

print(f"Variables renombradas: {len(existing_mappings)}")
print(f"Nuevos nombres de columnas:")
for old, new in existing_mappings.items():
    print(f"  {old} -> {new}")

print(f"\nDimensiones finales del dataset: {df.shape}")
print(f"Columnas finales: {list(df.columns)}")

Variables renombradas: 19
Nuevos nombres de columnas:
  customerID -> id_cliente
  Churn -> abandono
  gender -> genero
  SeniorCitizen -> mayor_65
  Partner -> tiene_pareja
  Dependents -> tiene_dependientes
  tenure -> antiguedad_meses
  PhoneService -> servicio_telefono
  MultipleLines -> lineas_multiples
  InternetService -> servicio_internet
  OnlineSecurity -> seguridad_online
  OnlineBackup -> respaldo_online
  DeviceProtection -> proteccion_dispositivo
  TechSupport -> soporte_tecnico
  StreamingTV -> streaming_tv
  StreamingMovies -> streaming_peliculas
  Contract -> tipo_contrato
  PaperlessBilling -> facturacion_digital
  PaymentMethod -> metodo_pago

Dimensiones finales del dataset: (7043, 22)
Columnas finales: ['id_cliente', 'abandono', 'genero', 'mayor_65', 'tiene_pareja', 'tiene_dependientes', 'antiguedad_meses', 'servicio_telefono', 'lineas_multiples', 'servicio_internet', 'seguridad_online', 'respaldo_online', 'proteccion_dispositivo', 'soporte_tecnico', 'streaming_tv'

#📊 Carga y análisis

#📄Informe final